In [1]:
import pandas as pd
import numpy as np

# Task 2

I download data from IPUMS-USA, from year 2001 onwards. I use the following variables:
- Demographics: `SEX`, `AGE`, `RACE`, `MARST`, `NCHILD`, `EDUC` (and detailed version `EDUCD`)
- Geographic: `STATEFIP`, `COUNTYFIP`, `METAREA`
- `TRANTIME`
- Work variables: `OCC2010`, `IND1990`, `EMPSTAT`, `LABFORCE`, `UHRSWORK` (as close as I can get to `UHRSWORK1`), `INCWAGE` (as close as I can get to `EARNWEEK`)

In [2]:
data = pd.read_csv('census_data.csv')

## Data cleaning 

Here I remove some NA values for the variables used in the next step. Specifically, I remove:
- `0` for `TRANTIME`
- `1` for `EDUCD` (this is transmitted to `EDUC` as well)

I also recode `MARST` variable, specifically:
- `1, 2` are recoded to `married`
- `3, 4, 5, 6` are recoded to `single`

After that, I create a new variable `educ_level`, which collapses the `EDUC` values as follows:
- `EDUC < 7` becomes `Less than college`
- `7 <= EDUC <= 9` becomes `Some college` (NB: this also includes persons with 2 year Associate's degrees)
- `EDUC == 10` becomes `College/BA`
- `EDUC == 11` becomes `More than BA`

Finally, I create a variable `has_child`, which is coded as 0 if `NCHILD` (number of children currently in household) is 0, and 1 if `NHCHILD > 0`.

In [3]:
# NA variable name/values pairs
NA_values = {
    'TRANTIME': [0],
    'EDUCD': [1],
}

# Remove NA values
for variable, nas in NA_values.items():
    data.loc[data[variable].isin(nas), variable] = None
    
# Match EDUCD NAs to EDUC NAs
data.loc[data['EDUCD'].isna(), 'EDUC'] = None
    
# Key for replacing MARST values
marst_replace = {
    1: 'married',
    2: 'married',
    3: 'single',
    4: 'single',
    5: 'single',
    6: 'single',
}

# replace MARST values
data['MARST'] = data['MARST'].replace(marst_replace)

# Create educ_level
data['educ_level'] = None
data.loc[data['EDUC'].between(0,6) , 'educ_level'] = 'Less than college'
data.loc[data['EDUC'].between(7,9) , 'educ_level'] = 'Some college'
data.loc[data['EDUC'] == 10 , 'educ_level'] = 'College/BA'
data.loc[data['EDUC'] == 11 , 'educ_level'] = 'More than BA'

# Create had_child
data['has_child'] = None
data.loc[data['NCHILD'] == 0, 'has_child'] = 0
data.loc[data['NCHILD'] > 0, 'has_child'] = 1

## Commuting time

First I restrict the sample to `LABFORCE == 2` (in the labor force) and `AGE` between 25 and 55. I also drop all entries with NA values in any of `MARST`, `SEX`, `EDUC` or `TRANTIME`.

Then I compute a pivot table, with values of `MARST` and `SEX` in the columns, and `EDUC` in the rows, with the value being the weighted mean of `TRANTIME`, weighted using the `PERWT` column.

In [4]:
data = data.query('LABFORCE == 2 & AGE >= 25 & AGE <= 55').copy()
data = data.dropna(subset = ['MARST', 'SEX', 'EDUC', 'TRANTIME'])

In [5]:
# Helper function for computing the weighted mean (weight and value column name fixed)
weight_col = 'PERWT'
vales_col = 'TRANTIME'

# Main function
def weighted_mean(data):
    return np.average(data[vales_col], weights = data[weight_col])

In [6]:
# Split by collapsed educ categories, save
table = data.groupby(['SEX', 'MARST', 'educ_level']).apply(weighted_mean).unstack()
table.to_csv('Mean commute times.csv')

# Show result
table

educ_level   College/BA  Less than college  More than BA  Some college
SEX MARST                                                             
1   married   29.539546          28.315639     28.509616     28.486393
    single    26.985904          26.918012     26.321075     26.312109
2   married   25.340071          22.396347     26.004190     23.846065
    single    26.693739          24.040448     26.783194     25.108546

In [7]:
# Split by has_child, save
table_c = data.groupby(['SEX', 'has_child', 'MARST', 'educ_level']).apply(weighted_mean).unstack()
table_c.to_csv('Mean commute times - child.csv')

# Show result
table_c

educ_level             College/BA  Less than college  More than BA  \
SEX has_child MARST                                                  
1   0         married   28.536589          27.765056     27.539720   
              single    26.814500          26.746350     26.112471   
    1         married   29.942864          28.525276     28.863180   
              single    28.496999          27.465533     28.109670   
2   0         married   26.279064          22.881645     26.866324   
              single    26.610004          24.244894     26.663779   
    1         married   24.877931          22.172656     25.572238   
              single    26.919843          23.867963     27.146852   

educ_level             Some college  
SEX has_child MARST                  
1   0         married     27.857758  
              single      26.071323  
    1         married     28.732178  
              single      27.394377  
2   0         married     24.508201  
              single      25.121938  
    1         married     23.558255  
              single      25.094532